In [2]:
pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = 'Fact_Customer_Orders'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [24]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [25]:
data = df[['PRODUCT_ID','ORDER_QTY','ORDER_DATE']]

In [26]:
#Convert order_date to datetime format
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [28]:
#Aggregate data by date and sum of quantities per day
data_daily= data.groupby('ORDER_DATE').agg({'ORDER_QTY':'sum'}).reset_index()

In [30]:
#set order_date as index required for time series analysis
data_daily.set_index('ORDER_DATE',inplace=True)

In [44]:
data_daily

,ORDER_QTY
ORDER_DATE,
2019-01-02,11600
2019-01-03,12150
2019-01-04,10100
2019-01-05,104650
2019-01-06,109250
...,...
2024-08-26,66850
2024-08-27,63650
2024-08-28,73200


In [45]:
data_daily.sort_index(inplace=True)

In [46]:
data_daily

,ORDER_QTY
ORDER_DATE,
2019-01-02,11600
2019-01-03,12150
2019-01-04,10100
2019-01-05,104650
2019-01-06,109250
...,...
2024-08-26,66850
2024-08-27,63650
2024-08-28,73200


In [50]:
# Group data by 'product_id'
grouped_data = data_daily.groupby('PRODUCT_ID')

KeyError: 'PRODUCT_ID'

In [48]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'test_forecast_order_qty','rse', 'rmse','r2_score'])

In [49]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_DATE', 'forecast_order_qty'])